<a href="https://colab.research.google.com/github/Diwakarmogarala/Capstone/blob/main/Hospital_Price_Prediction/Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://drive.google.com/drive/u/0/folders/11zhBUqnUT6J4IVMLpBcrgz8saXAGJ7Hi

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Capstone Project/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2016_20250531.csv")

<ipython-input-3-ea0d73f45d84>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/Capstone Project/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2016_20250531.csv")


In [5]:
pd.set_option('display.max_columns', None)
df.head()

,Health Service Area,Hospital County,Operating Certificate Number,Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Length of Stay,Type of Admission,Patient Disposition,Discharge Year,CCS Diagnosis Code,CCS Diagnosis Description,CCS Procedure Code,CCS Procedure Description,APR DRG Code,APR DRG Description,APR MDC Code,APR MDC Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Abortion Edit Indicator,Emergency Department Indicator,Total Charges,Total Costs
0,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,70 or Older,147,F,White,Not Span/Hispanic,3,Urgent,Home or Self Care,2016,122,Pneumonia (except that caused by tuberculosis ...,0,NO PROC,139,Other pneumonia,4,Diseases and Disorders of the Respiratory System,2,Moderate,Moderate,Medical,Medicare,Private Health Insurance,NaN,0,N,N,3913.23,3466.83
1,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,30 to 49,147,M,White,Not Span/Hispanic,2,Elective,Home or Self Care,2016,122,Pneumonia (except that caused by tuberculosis ...,0,NO PROC,139,Other pneumonia,4,Diseases and Disorders of the Respiratory System,1,Minor,Minor,Medical,Self-Pay,NaN,NaN,0,N,Y,3597.10,3746.80
2,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,50 to 69,147,F,White,Not Span/Hispanic,7,Urgent,Home or Self Care,2016,127,Chronic obstructive pulmonary disease and bron...,0,NO PROC,140,Chronic obstructive pulmonary disease,4,Diseases and Disorders of the Respiratory System,1,Minor,Minor,Medical,Medicare,Federal/State/Local/VA,NaN,0,N,Y,9149.22,8322.10
3,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,70 or Older,147,M,White,Not Span/Hispanic,4,Urgent,Home or Self Care,2016,122,Pneumonia (except that caused by tuberculosis ...,0,NO PROC,139,Other pneumonia,4,Diseases and Disorders of the Respiratory System,2,Moderate,Moderate,Medical,Medicare,Medicaid,NaN,0,N,Y,5880.61,5828.87
4,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,50 to 69,147,M,White,Not Span/Hispanic,5,Elective,Home or Self Care,2016,122,Pneumonia (except that caused by tuberculosis ...,202,ELECTROCARDIOGRAM,139,Other pneumonia,4,Diseases and Disorders of the Respiratory System,2,Moderate,Moderate,Medical,Blue Cross/Blue Shield,NaN,NaN,0,N,Y,7019.76,6604.00


In [9]:
df["Payment Typology 1"].value_counts()

,count
Payment Typology 1,
Medicare,884212
Medicaid,723774
Private Health Insurance,333581
Blue Cross/Blue Shield,271620
Self-Pay,54795
Miscellaneous/Other,27758
"Managed Care, Unspecified",20288
Federal/State/Local/VA,16576
Unknown,6896


In [11]:
df["APR MDC Description"].value_counts()

,count
APR MDC Description,
Diseases and Disorders of the Circulatory System,293156
"Pregnancy, Childbirth and the Puerperium",250210
Newborns and Other Neonates with Conditions Originating in the Perinatal Period,229534
Diseases and Disorders of the Musculoskeletal System and Conn Tissue,204509
Diseases and Disorders of the Digestive System,195919
Diseases and Disorders of the Respiratory System,193933
Diseases and Disorders of the Nervous System,151206
"Infectious and Parasitic Diseases, Systemic or Unspecified Sites",134120
Mental Diseases and Disorders,115050


In [15]:
insurance_dic = {}
df[df["Payment Typology 1"]=="Medicare"]["APR MDC Description"].value_counts()
for insurance in df["Payment Typology 1"].value_counts().index:
  insurance_dic[insurance]=df[df["Payment Typology 1"]==insurance]["APR MDC Description"].value_counts()
insurance_df = pd.DataFrame(insurance_dic)

insurance_df

,Medicare,Medicaid,Private Health Insurance,Blue Cross/Blue Shield,Self-Pay,Miscellaneous/Other,"Managed Care, Unspecified",Federal/State/Local/VA,Unknown,Department of Corrections
APR MDC Description,,,,,,,,,,
Alcohol/Drug Use and Alcohol/Drug Induced Organic Mental Disorders,9556.0,51129,6430,3843,3787,126.0,274,314.0,445.0,115.0
Burns,351.0,792,263,203,80,177.0,7,11.0,1.0,7.0
"Diseases and Disorders of Blood, Blood Forming Organs and Immunological Disorders",12961.0,11753,3854,2970,693,176.0,351,188.0,105.0,98.0
Diseases and Disorders of the Circulatory System,180611.0,50496,27397,23517,4883,1267.0,1620,1560.0,1236.0,569.0
Diseases and Disorders of the Digestive System,88946.0,46201,27304,24106,4267,694.0,2181,1191.0,575.0,454.0
Diseases and Disorders of the Eye,1193.0,1203,474,369,162,102.0,39,20.0,8.0,25.0
Diseases and Disorders of the Female Reproductive System,4777.0,7485,6625,5023,694,46.0,357,229.0,37.0,10.0
Diseases and Disorders of the Hepatobiliary System and Pancreas,23668.0,19984,9458,8018,1828,247.0,781,531.0,166.0,112.0
Diseases and Disorders of the Kidney and Urinary Tract,62764.0,19239,8983,7927,1773,401.0,721,510.0,264.0,148.0


In [21]:
plt.figure(figsize=(100,100))
sns.barplot(data=insurance_df)

<Axes: >

In [20]:
df["APR DRG Description"].value_counts()

,count
APR DRG Description,
"Neonate birthwt >2499g, normal newborn or neonate w other problem",195238
Vaginal delivery,142275
Septicemia & disseminated infections,93349
Cesarean delivery,74561
Heart failure,56708
...,...
Pancreas transplant,47
Extensive 3rd degree burns w skin graft,47
Cleft lip & palate repair,46


In [22]:
drug_dic = {}
df[df["Payment Typology 1"]=="Medicare"]["APR DRG Description"].value_counts()
for insurance in df["Payment Typology 1"].value_counts().index:
  drug_dic[insurance]=df[df["Payment Typology 1"]==insurance]["APR DRG Description"].value_counts()
drug_df = pd.DataFrame(drug_dic)

drug_df

,Medicare,Medicaid,Private Health Insurance,Blue Cross/Blue Shield,Self-Pay,Miscellaneous/Other,"Managed Care, Unspecified",Federal/State/Local/VA,Unknown,Department of Corrections
APR DRG Description,,,,,,,,,,
Abdominal pain,2109.0,2249,858,743,196.0,38.0,78.0,49.0,11.0,21.0
"Abortion w/o D&C, aspiration curettage or hysterotomy",13.0,407,91,85,28.0,2.0,3.0,1.0,NaN,NaN
Acute & subacute endocarditis,268.0,188,62,60,11.0,6.0,3.0,4.0,1.0,2.0
Acute anxiety & delirium states,891.0,1222,539,343,120.0,29.0,28.0,31.0,27.0,14.0
Acute leukemia,719.0,403,263,203,19.0,8.0,30.0,14.0,2.0,5.0
...,...,...,...,...,...,...,...,...,...,...
Vaginal delivery w sterilization &/or D&C,33.0,2016,609,523,49.0,4.0,33.0,63.0,1.0,NaN
Ventricular shunt procedures,526.0,289,205,187,16.0,10.0,18.0,8.0,1.0,NaN
Vertigo & other labyrinth disorders,3017.0,1005,486,445,104.0,14.0,40.0,24.0,7.0,4.0
